<a href="https://colab.research.google.com/github/FKz11/Neural_Network/blob/main/lesson-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## lesson-3

Библиотеки

In [ ]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

import matplotlib
import matplotlib.pyplot as plt

import time

import warnings

In [ ]:
print(tf.__version__)
msg = tf.constant('TensorFlow 2.0 Hello World')
tf.print(msg)

2.8.0
TensorFlow 2.0 Hello World


In [ ]:
x = tf.random.uniform([3, 3])

print("есть ли доступные GPU: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("тензор на GPU #0:  "),
print(x.device.endswith('GPU:0'))

есть ли доступные GPU: 
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
тензор на GPU #0:  
True


In [ ]:
def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 циклов: {:0.2f}ms".format(1000*result))

# исполнение CPU
print(" CPU работает:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# исполнение на GPU #0 , если доступен
if tf.config.experimental.list_physical_devices("GPU"):
  print(" какой GPU:")
  with tf.device("GPU:0"): # или GPU:1 , для 2х GPU, GPU:2 для 3х  и т.д.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

 CPU работает:
10 циклов: 124.00ms
 какой GPU:
10 циклов: 2.00ms


### 1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора) на датасет from sklearn.datasets import load_boston.

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    X, y = load_boston(return_X_y=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3333, shuffle=True, random_state=42)

In [ ]:
def nn(X_train, X_test, y_train, y_test, neuron = 100, layer = 2, optimizer='adam', 
                                   loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], seed_list = range(5)):
    
    metrics_train = []
    metrics_test = []
    
    for seed in seed_list:
        tf.random.set_seed(seed)
        model = Sequential(name='my_model')
        model.add(Dense(    neuron, activation='tanh', input_shape=(13,)))  
        # входной слой + поносвязный слой с активацией ReLU
        for k in range(layer - 1):
          model.add(Dense(  neuron, activation='tanh')) 
          # Скрытый слой k из neuron_numb нейронов с активацией tanh

        model.add(Dense(1, activation='relu')) 
        # выходной слой из 10 нейронов (сколько классов, столько нейронов) + активация relu
        
        model.compile(
          optimizer=optimizer, # оптимизатор
          loss=loss, # функция потерь
          metrics=metrics, # метрика
        ) 
        
        with tf.device("GPU:0"):
            # Train the model.
            hh = model.fit(    
                X_train,
                y_train,
                epochs=15,
                batch_size=32, verbose=0
            )

            # Evaluate the model.
            eval_test = model.evaluate( 
                X_test,
                y_test, verbose=0
            )
        
        # print([hh.history[key][-1] for key in hh.history][1:], eval_test[1:])
        metrics_train.append([hh.history[key][-1] for key in hh.history][1:])
        metrics_test.append(eval_test[1:])
        
    return (dict(zip(list(hh.history.keys())[1:], np.mean(metrics_train, axis=0))), 
            dict(zip(list(hh.history.keys())[1:], np.mean(metrics_test, axis=0))))

In [ ]:
%%time

nn(X_train, X_test, y_train, y_test, neuron = 300, layer = 3, optimizer='adam', 
                           loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()], seed_list = range(5))

Wall time: 15.4 s


({'root_mean_squared_error': 7.9063401222229},
 {'root_mean_squared_error': 7.521875190734863})

При некоторых seed модель бывает редко расходится, я пока не нашёл что нужно настроить, чтобы она не ожиданно не расходилась, поэтому я решил зафиксировать seed = 0, 1, 2, 3, 4. Чтобы дальше эксперименты были +- честные.

### 2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

Замена метрик никак не влияет на результат. Поэтому будем менять функции потерь и считать все метрики.

In [ ]:
%%time

loss_list = [tf.keras.losses.MeanSquaredError(), tf.keras.losses.MeanAbsoluteError(), 
             tf.keras.losses.MeanAbsolutePercentageError(), tf.keras.losses.MeanSquaredLogarithmicError(), 
             tf.keras.losses.CosineSimilarity(), tf.keras.losses.Huber(), tf.keras.losses.LogCosh()]

metrics_list = [tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError(), 
                tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsolutePercentageError(),
                tf.keras.metrics.MeanSquaredLogarithmicError(), tf.keras.metrics.CosineSimilarity(),
                tf.keras.metrics.LogCoshError(), tfa.metrics.RSquare(y_shape=(1,))]

metrics_train_loss = []
metrics_test_loss = []

for loss in loss_list:
    metrics_loss = nn(X_train, X_test, y_train, y_test, neuron = 300, layer = 3, optimizer='adam', 
                      loss=loss, metrics=metrics_list, seed_list = range(5))
    print(f'{loss.name} - {metrics_loss}\n')
    metrics_train_loss.append(metrics_loss[0])
    metrics_test_loss.append(metrics_loss[1])

mean_squared_error - ({'mean_squared_error': 62.52296600341797, 'mean_absolute_error': 5.763467597961426, 'root_mean_squared_error': 7.9063401222229, 'mean_absolute_percentage_error': 27.297187423706056, 'mean_squared_logarithmic_error': 0.09534284174442291, 'cosine_similarity': 1.0, 'logcosh': 5.111546421051026, 'r_square': 0.292223846912384}, {'mean_squared_error': 56.606136322021484, 'mean_absolute_error': 5.785540771484375, 'root_mean_squared_error': 7.521875190734863, 'mean_absolute_percentage_error': 30.48138999938965, 'mean_squared_logarithmic_error': 0.10608127117156982, 'cosine_similarity': 1.0, 'logcosh': 5.127929019927978, 'r_square': 0.24747884273529053})

mean_absolute_error - ({'mean_squared_error': 66.03183212280274, 'mean_absolute_error': 5.268998622894287, 'root_mean_squared_error': 8.122534275054932, 'mean_absolute_percentage_error': 22.807318878173827, 'mean_squared_logarithmic_error': 0.09179328829050064, 'cosine_similarity': 1.0, 'logcosh': 4.646097946166992, 'r_sq

Таблица для трейна:

In [ ]:
result_train_loss = pd.DataFrame(metrics_train_loss, index = [['loss']*len(loss_list), list(map(lambda x: x.name, loss_list))])
result_train_loss

mean_squared_error  mean_absolute_error  \
loss mean_squared_error                       62.522966             5.763468   
     mean_absolute_error                      66.031832             5.268999   
     mean_absolute_percentage_error           75.567773             5.607187   
     mean_squared_logarithmic_error           70.594029             5.617382   
     cosine_similarity                       471.725989            19.664167   
     huber_loss                               63.835774             5.150783   
     log_cosh                                 63.607452             5.227154   

                                     root_mean_squared_error  \
loss mean_squared_error                             7.906340   
     mean_absolute_error                            8.122534   
     mean_absolute_percentage_error                 8.690640   
     mean_squared_logarithmic_error                 8.401350   
     cosine_similarity                             21.695811   
     huber_loss                                     7.983922   
     log_cosh                                       7.972030   

                                     mean_absolute_percentage_error  \
loss mean_squared_error                                   27.297187   
     mean_absolute_error                                  22.807319   
     mean_absolute_percentage_error                       22.653103   
     mean_squared_logarithmic_error                       24.355755   
     cosine_similarity                                    83.582907   
     huber_loss                                           21.916835   
     log_cosh                                             22.451482   

                                     mean_squared_logarithmic_error  \
loss mean_squared_error                                    0.095343   
     mean_absolute_error                                   0.091793   
     mean_absolute_percentage_error                        0.106853   
     mean_squared_logarithmic_error                        0.098845   
     cosine_similarity                                     2.992739   
     huber_loss                                            0.086596   
     log_cosh                                              0.087283   

                                     cosine_similarity    logcosh  r_square  
loss mean_squared_error                       1.000000   5.111546  0.292224  
     mean_absolute_error                      1.000000   4.646098  0.252503  
     mean_absolute_percentage_error           1.000000   4.987116  0.144553  
     mean_squared_logarithmic_error           1.000000   4.983841  0.200857  
     cosine_similarity                        0.989318  18.971138 -4.340061  
     huber_loss                               1.000000   4.529148  0.277363  
     log_cosh                                 1.000000   4.600628  0.279947

Таблица для теста:

In [ ]:
result_test_loss = pd.DataFrame(metrics_test_loss, index = [['loss']*len(loss_list), list(map(lambda x: x.name, loss_list))])
result_test_loss

mean_squared_error  mean_absolute_error  \
loss mean_squared_error                       56.606136             5.785541   
     mean_absolute_error                      48.383141             4.637920   
     mean_absolute_percentage_error           54.622099             4.628236   
     mean_squared_logarithmic_error           56.574421             5.112848   
     cosine_similarity                       400.363116            18.106251   
     huber_loss                               48.436947             4.593945   
     log_cosh                                 48.961426             4.650393   

                                     root_mean_squared_error  \
loss mean_squared_error                             7.521875   
     mean_absolute_error                            6.949594   
     mean_absolute_percentage_error                 7.389369   
     mean_squared_logarithmic_error                 7.520710   
     cosine_similarity                             19.986327   
     huber_loss                                     6.956135   
     log_cosh                                       6.996003   

                                     mean_absolute_percentage_error  \
loss mean_squared_error                                   30.481390   
     mean_absolute_error                                  22.833334   
     mean_absolute_percentage_error                       21.445618   
     mean_squared_logarithmic_error                       25.571127   
     cosine_similarity                                    81.943355   
     huber_loss                                           22.096838   
     log_cosh                                             22.767510   

                                     mean_squared_logarithmic_error  \
loss mean_squared_error                                    0.106081   
     mean_absolute_error                                   0.079518   
     mean_absolute_percentage_error                        0.085139   
     mean_squared_logarithmic_error                        0.094221   
     cosine_similarity                                     2.684794   
     huber_loss                                            0.078381   
     log_cosh                                              0.081064   

                                     cosine_similarity    logcosh  r_square  
loss mean_squared_error                            1.0   5.127929  0.247479  
     mean_absolute_error                           1.0   4.018040  0.356795  
     mean_absolute_percentage_error                1.0   4.025768  0.273855  
     mean_squared_logarithmic_error                1.0   4.478338  0.247900  
     cosine_similarity                             1.0  17.414212 -4.322421  
     huber_loss                                    1.0   3.972213  0.356080  
     log_cosh                                      1.0   4.029663  0.349108

Сравним все модели на тесте с помощью метрики r2

In [ ]:
sorted(zip(map(lambda x: x.name, loss_list), map(lambda x: x['r_square'], metrics_test_loss)), key=lambda i: i[1], reverse=True)

[('mean_absolute_error', 0.3567952156066895),
 ('huber_loss', 0.3560799241065979),
 ('log_cosh', 0.3491075158119202),
 ('mean_absolute_percentage_error', 0.2738546133041382),
 ('mean_squared_logarithmic_error', 0.24790046215057374),
 ('mean_squared_error', 0.24747884273529053),
 ('cosine_similarity', -4.322421455383301)]

Видно, что модели с ошибками mean_absolute_error, huber_loss и log_cosh справились лучше всех. mean_absolute_percentage_error, mean_squared_logarithmic_error и mean_squared_error справились хуже. модель с cosine_similarity вообще разошлась.

Метрика на тесте получилась намного больше чем на трейне, что очень странно.

Так же модель с loss = MAE, показала себя лучше всех, а с loss = MSE хуже всех из тех которые сошлись.

Из этого можно предположить, что в X_train есть выбросы относительно X_test. Поэтому модель с loss = MAE меньше обращала на них внимание и показала метрику на тесте лучше чем модель с loss = MSE, которая сильнее обращала внимание на предпологаемые выбросы. Поэтому же модели показали метрику на тесте лучше, чем на трейне (из-за предположительных выбросов в трейне). Хотя в учебных датасетах выбросов вроде бы быть не должно. Так что наверное такой результат показатель сильной зависимости нейронной сети от разбиения датасета. 

### 3. Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

Я нашёл аддоны к tensorflow, это tensorflow_addons, там очень много всего крутого и немного скрытно, поэтому на соревнованиях или где то еще может быть очень полезным. Я взял оттуда метрику r2 - tfa.metrics.RSquare(y_shape=(1,)), тут нужно было в ручнчую указывать какого размера данные к нам придут (y_shape), что сначала стало неожиданностью.

In [ ]:
matrix = tf.constant([1,2,3,4,5,6,7,8,9], shape=[3,3])
matrix

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])>

#### tf.cast

Функция, которая приводит тензор к определённому типу данных. Очень полезная функция, так как для большинства вычислений тип данных должен быть не int а float.

In [ ]:
matrix = tf.cast(matrix, tf.float32)
matrix

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32)>

#### tf.linalg.svd

Функция, которая находит сингулярное разложение тензора (находит сингулярное разложение каждой матрицы в последовательных данных). Может применяться для сжатия данных, так как сингулярное разложение наилучшим образом сохраняет информативность сжимая матрицу путям отбрасывания наименьших сингулярных чисел (снизу) в диагональной матрице s (здесь она представленна в виде вектора диагонали).

In [ ]:
s, u, v = tf.linalg.svd(matrix)
s, u, v

(<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1.6848103e+01, 1.0683688e+00, 1.7066353e-07], dtype=float32)>,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[-0.21483709,  0.88723046,  0.40824854],
        [-0.5205874 ,  0.24964422, -0.8164965 ],
        [-0.8263375 , -0.38794297,  0.40824816]], dtype=float32)>,
 <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[-0.47967124, -0.77669084,  0.4082485 ],
        [-0.5723677 , -0.07568672, -0.81649655],
        [-0.6650644 ,  0.6253182 ,  0.40824807]], dtype=float32)>)

#### tf.linalg.diag

Функция, которая строит матрицу по её диагонали, так как таскать везде матрицу, где есть только диагональные элементы может быть весьма затратно, то её таскают в виде вектора, но для дальнейших вычислений её нужно преобразовать обратно в матрицу.

In [ ]:
s = tf.linalg.diag(s)
s

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1.6848103e+01, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0683688e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.7066353e-07]], dtype=float32)>

#### tf.transpose

Функция транспонирования. Очень полезна, так как это задача часто встречается.

In [ ]:
matrix_usv = u @ s @ tf.transpose(v)
matrix_usv

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.9999997, 1.999998 , 2.9999976],
       [4.       , 4.999999 , 5.9999995],
       [7.0000005, 7.9999986, 8.999999 ]], dtype=float32)>

#### tf.round

Функция округления. Может быть полезно для красивых отчетов и не только.

In [ ]:
matrix_usv = tf.round(matrix_usv)
matrix_usv

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]], dtype=float32)>